In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth
from folium import GeoJson
import geopandas as gpd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [2]:
pd.set_option('display.max_columns', None)

df1 = pd.read_csv('../bin/data1.csv')
df2 = pd.read_csv('../bin/data2.csv')
df3 = pd.read_csv('../bin/data3.csv')

df = pd.concat([df1, df2, df3], ignore_index=True)

df.head()

,provincia,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carroceria,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion
0,Madrid,Detalles BMW Serie 3 (2018),Matriculado: 11/2018,58000,RUTA 66,Profesional,C,12 meses,19990,37250,4633 mm Largo,1811 mm Ancho,1429 mm Alto,Capacidad del maletero (l)480/0,CarroceríaBerlina,Número de plazas5,Batalla (mm)\r\n2.810,Peso/Masa max. autorizado (kg)2.045,Número de puertas4,"Consumo medio 4,50 litros/100 km","Consumo en carretera 4,00 litros/100 km","Consumo urbano 5,30 litros/100 km",CO2 (g/km) 118 g/km,"Depósito de combustible (l) 57,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.995,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Manual,TracciónTracción trasera,Número de marchas6,Potencia (KW)110,Potencia (CV)150,Par (Nm)320,Velocidad máxima (km/h)215,"Aceleración 0-100 km/h (s)8,70"
1,Madrid,Detalles FIAT 500 (2021),Matriculado: 12/2021,34680,RUTA 66,Profesional,ECO,12 meses,12990,19300,3571 mm Largo,1627 mm Ancho,1488 mm Alto,Capacidad del maletero (l)185/0,CarroceríaBerlina,Número de plazas4,Batalla (mm)\r\n2.300,Peso/Masa max. autorizado (kg)1.055,Número de puertas3,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 35,00 litros",Combustible Gasolina,Nº de cilindros 3 cilindros,Cilindrada (cm3) 999,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)52,Potencia (CV)70,Par (Nm)92,Velocidad máxima (km/h)167,"Aceleración 0-100 km/h (s)13,80"
2,Madrid,Detalles NISSAN Qashqai (2017),Matriculado: 04/2017,66933,AUTOMOTOR DURSAN,Profesional,C,12 meses,17390,26650,4377 mm Largo,1806 mm Ancho,1590 mm Alto,Capacidad del maletero (l)430/1.598,CarroceríaTodo Terreno,Número de plazas5,Batalla (mm)\r\n2.646,Peso/Masa max. autorizado (kg)1.910,Número de puertas5,"Consumo medio 3,80 litros/100 km","Consumo en carretera 3,60 litros/100 km","Consumo urbano 4,20 litros/100 km",CO2 (g/km) 103 g/km,"Depósito de combustible (l) 55,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.461,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)81,Potencia (CV)110,Par (Nm)260,Velocidad máxima (km/h)182,"Aceleración 0-100 km/h (s)11,90"
3,Sevilla,Detalles OPEL Astra (2017),Matriculado: 01/2017,159000,Mules Car,Profesional,C,12 meses,8000,22425,4702 mm Largo,1809 mm Ancho,1499 mm Alto,Capacidad del maletero (l)540/1.630,CarroceríaStationwagon,Número de plazas5,Batalla (mm)\r\n2.662,Peso/Masa max. autorizado (kg)1.985,Número de puertas5,"Consumo medio 3,60 litros/100 km","Consumo en carretera 3,30 litros/100 km","Consumo urbano 4,20 litros/100 km",CO2 (g/km) 99 g/km,"Depósito de combustible (l) 48,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.598,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)81,Potencia (CV)110,Par (Nm)300,Velocidad máxima (km/h)195,"Aceleración 0-100 km/h (s)11,40"
4,Madrid,Detalles MITSUBISHI Eclipse Cross (2022),Matriculado: 07/2022,62695,CLICARS MADRID,Profesional,0 EMISIONES,12 meses,22990,44700,4545 mm Largo,1805 mm Ancho,1685 mm Alto,Capacidad del maletero (l)404/1.153,CarroceríaTodo Terreno,Número de plazas5,Batalla (mm)\r\n2.670,Peso/Masa max. autorizado (kg)2.425,Número de puertas5,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 43,00 litros",Combustible Gasolina y corriente eléctrica,Nº de cilind

## Limpieza preliminar

In [4]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
22,CO2,236,29.562183,29.562183,object
35,aceleracion,145,29.497669,29.497669,object
33,par,233,29.494736,29.494736,object
34,velocidad_max,157,29.491804,29.491804,object
16,batalla,346,29.491804,29.491804,object
13,capacidad_maletero,929,29.491804,29.491804,object
31,potencia_kw,248,29.403830,29.403830,object
23,deposito,88,29.400897,29.400897,object
25,cilindros,11,29.400897,31.653031,object
12,alto,431,29.400897,29.400897,object


In [5]:
df['tipo_cambio'].unique()

#Hay nans que se toman como floats. Lo convierto a "no disponible"
df['tipo_cambio'] = df['tipo_cambio'].fillna("no disponible")

#No sé qué es manual automatizada, si queremos que sea manual hay que poner la lambda al revés
df['tipo_cambio'] = df['tipo_cambio'].apply(lambda x: 'Manual' if 'Manual' in x else 'Automático')

In [6]:
df["carroceria"] = df["carroceria"].str.replace("Carrocería","")

df["combustible"] = df["combustible"].str.replace("combustible ","").str.replace("Combustible ","")

In [7]:
df['fecha_matriculacion'] = df['fecha_matriculacion'].str.split("Matriculado: ").str[1]

def limpiar_fecha(fecha):
    try:
        mes, ano = fecha.split('/')
        return int(mes), int(ano)
    except:
        return np.nan, int(fecha)

df[['mes_matriculacion', 'ano_matriculacion']] = df['fecha_matriculacion'].apply(lambda x: pd.Series(limpiar_fecha(x)))

df = df.drop(columns=['fecha_matriculacion'])

In [8]:
df['mes_matriculacion'] = df['mes_matriculacion'].fillna(0).astype('int64')
df['ano_matriculacion'] = df['ano_matriculacion'].fillna(0).astype('int64')

In [9]:
df = df[df["tipo_vendedor"] == "Profesional"]
df = df.drop(columns=['tipo_vendedor'])

In [10]:
df["garantia"] = df["garantia"].str.split(' ').str[0].replace("Sí",np.nan).replace("No",0).astype(float)

In [11]:
df["precio_contado"] = pd.to_numeric(df["precio_contado"], errors='coerce')

df["precio_nuevo"] = pd.to_numeric(df["precio_nuevo"], errors='coerce')

df['largo'] = pd.to_numeric(df['largo'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['ancho'] = pd.to_numeric(df['ancho'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['alto'] = pd.to_numeric(df['alto'].str.extract('(\d+\.?\d*)')[0], errors='coerce')

df['capacidad_maletero'] = pd.to_numeric(df['capacidad_maletero'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
df["num_plazas"] = df["num_plazas"].str.split("s").str[1].astype("Int64")
df["batalla"] = df["batalla"].str.replace('Batalla (mm)\r\n', '').str.replace('.','').astype("Int64")
df["peso"] = df["peso"].str.replace('Peso/Masa max. autorizado (kg)', '').str.replace('.','').astype("Int64")
df["num_puertas"] = df["num_puertas"].str.extract(r'(\d+)').astype("Int64")
df["consumo_medio"] = df["consumo_medio"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_carretera"] = df["consumo_carretera"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_urbano"] = df["consumo_urbano"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["co2"] = df["CO2"].str.extract(r'(\d{1,3})(?= g/km)')[0].astype(float)
df["num_cilindros"] = df["cilindros"].str.extract(r'(\d+)').astype("Int64")
df["cilindrada"] = df["cilindrada"].str.extract(r'(\d+,\d+|\d+\.\d+)', expand=False).str.replace(',', '').str.replace('.', '').astype("Int64")
df["deposito"] = df["deposito"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

df["num_marchas"] = df["num_marchas"].str.extract(r'(\d+)').astype("Int64")
df["potencia_kw"] = df["potencia_kw"].str.extract(r'(\d+)').astype("Int64")
df["potencia_cv"] = df["potencia_cv"].str.extract(r'(\d+)').astype("Int64")
df["par"] = df["par"].str.extract(r'(\d+)').astype("Int64")
df["velocidad_max"] = df["velocidad_max"].str.extract(r'(\d+)').astype("Int64")
df["aceleracion"] = df["aceleracion"].str.extract(r'(\d+,\d+|\d+\.\d+)')[0].str.replace(",", ".").astype(float)
df['id_sobrealimentacion'] = df['sobrealimentacion'].apply(lambda x: x.replace('Sobrealimentación', '').replace('Tipo de sobrealimentador', 'no disponible').strip() if pd.notna(x) and x != '-' else 'no disponible')

In [12]:
df['id_sobrealimentacion'].unique()

array(['Turbo de geometría variable', 'Turbo', 'no disponible', '-',
       'Doble turbo', 'Compresor de raices'], dtype=object)

In [13]:
df["traccion"].unique()

df["traccion"] = df['traccion'].fillna("no disponible")
df["traccion"] = df["traccion"].str.replace('Tracción', '').str.replace('Todo terreno tracción', '').apply(lambda x: x.strip())

In [14]:
marca_listado = pd.read_csv('../bin/listado_marcas.csv')

lista_marcas = [marca.upper() for marca in marca_listado['nombre_marca'].tolist()]

df['marca_modelo'] = df['marca_modelo'].fillna('').str.replace("Detalles ", "").str.strip().str.upper()

df['marca'] = df['marca_modelo'].apply(lambda x: next((marca for marca in lista_marcas if marca in x), np.nan))
df['modelo'] = df.apply(lambda x: re.sub(r'\s*\(.*?\)', '', x['marca_modelo'].replace(x['marca'], "").strip()).strip() if pd.notnull(x['marca']) else x['marca_modelo'], axis=1)


In [15]:
df = df.drop(columns=['marca_modelo', 'CO2', 'cilindros', 'sobrealimentacion', 'carroceria'])

In [16]:
df.rename(columns={'distintivo_ambiental': 'id_distintivo_ambiental'
                  , 'marca': 'id_marca'
                  , 'modelo': 'id_modelo'
                  , 'nombre_vendedor': 'id_concesionario'
                  , 'provincia': 'id_provincia'
                  , 'traccion': 'id_traccion'}, inplace=True)

In [17]:
df['combustible'] = (
    df['combustible']
    .str.replace('-', 'no disponible')
    .str.replace('Diésel', 'Diesel')
    .str.replace('Gasolina y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Diesel y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Corriente eléctrica', 'Eléctrico', regex=False)
    .str.replace(r'\bHíbrido\b(?!.*Enchufable)', 'Híbrido Enchufable', regex=True)
)

In [18]:
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split('-')[0])

In [19]:
for columna in df.columns:
    df[columna] = df[columna].replace(["no disponible", "<NA>", "-", "0", 0, np.nan], None)
    

In [45]:
df = df[df['precio_contado'].notna()]

## Rtdo limpieza

In [39]:
df.sample(10)

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
31259,Murcia,12280,AUDI HUERTAS MOTOR MURCIA,C,12.0,24000.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,None,Manual,None,None,None,110,None,None,None,8,2023,None,None,None,AUDI,A1
20914,Barcelona,87784,AUTOHERO BARCELONA,C,12.0,11899.0,27765.0,4360.0,1834.0,1520.0,380.0,5,2670,1735,5,4.6,3.2,5.9,60.0,Gasolina,1199,Manual,delantera,6,96,130,230,210,10.8,7,2017,104.0,3,None,CITROEN,C4
32881,Madrid,101353,CLICARS MADRID,C,12.0,13490.0,None,None,None,None,None,None,None,1427,5,5.4,4.6,6.8,None,Gasolina,1330,Manual,None,None,None,140,None,None,None,8,2018,None,None,None,RENAULT,SCENIC
9589,Badajoz,17856,"MAVEN E HIJOS, concesionario oficial Ford",C,11.0,26500.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,1600,Automático,None,None,None,145,None,None,None,10,2022,None,None,None,NISSAN,JUKE
11422,Murcia,7100,MARMOCASION STORE,C,48.0,21390.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,None,Manual,None,None,None,120,None,None,None,3,2024,None,None,None,HYUNDAI,I30
17376,Madrid,53767,CLICARS MADRID,ECO,12.0,15990.0,22800.0,4228.0,1797.0,1566.0,536.0,5,2639,1741,5,6.6,5.6,8.1,48.0,Gasolina/gas,None,Manual,delantera,5,74,100,170,173,13.3,5,2021,107.0,3,Turbo,RENAULT,CAPTUR
15507,Madrid,167000,CAR10 Mostoles,ECO,12.0,None,None,None,None,None,None,None,None,1360,5,3.3,3.6,3.2,None,Híbrido Enchufable,1798,Automático,None,None,None,122,None,None,None,12,2020,None,None,None,TOYOTA,COROLLA
9411,Madrid,10,"GYATA, servicio oficial Ford",C,12.0,29490.0,33873.0,4550.0,1840.0,1675.0,506.0,5,2700,2020,5,None,None,None,56.0,Gasolina,1998,Manual,delantera,6,121,165,210,201,10.3,3,2023,None,4,Turbo,MAZDA,CX-5
22190,Madrid,130977,STAR MADRID,ECO,24.0,69990.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Híbrido Enchufable,None,Automático,None,None,None,320,None,None,None,7,2021,None,None,None,MERCEDES-BENZ,CLASE GLE
13206,Barcelona,14824,M Ocasión Barcelona,C,12.0,20100.0,24790.0,4205.0,1800.0,1565.0,374.0,5,2600,1770,5,None,None,None,50.0,Gasolina,None,Manual,delantera,6,88,120,172,181,11.5,5,2023,116.0,3,Turbo,HYUNDAI,KONA


In [47]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
31,co2,231,59.175045,59.175045,object
16,consumo_carretera,91,56.393874,56.393874,object
17,consumo_urbano,165,56.384478,56.384478,object
15,consumo_medio,129,51.329512,51.329512,object
33,id_sobrealimentacion,4,43.866078,43.866078,object
18,deposito,86,36.033073,36.033073,object
20,cilindrada,226,36.004886,36.004886,object
32,num_cilindros,8,35.738670,35.738670,object
27,velocidad_max,155,30.420621,30.420621,object
10,capacidad_maletero,386,30.276551,30.276551,object


In [49]:
df.to_csv('../bin/data_preprocess.csv', index=False)

## Gráficos

In [ ]:
plt.figure(figsize=(10, 6))
for tipo_combustible in df['combustible'].unique():
    subset = df[df['combustible'] == tipo_combustible]
    plt.scatter(subset['potencia_cv'], subset['precio_contado'], label=tipo_combustible, alpha=0.6)

# Configurar etiquetas y título
plt.xlabel('Potencia (cv)')
plt.ylabel('Precio')
plt.title('Relación entre Precio y Potencia por Tipo de Combustible')
plt.legend(title='Combustible')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['kilometraje'], df['precio_contado'], c=df['ano_matriculacion'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Año de Matriculación')

# Configurar etiquetas y título
plt.xlabel('Kilometraje')
plt.ylabel('Precio Contado')
plt.title('Relación entre Kilometraje y Precio Contado por Año de Matriculación')
plt.show()

In [ ]:
precio_medio = df.groupby('id_marca')[['precio_nuevo', 'precio_contado']].mean().sort_values(by='precio_contado', ascending=False)

precio_medio.plot(kind='bar', figsize=(18, 7))
plt.xlabel('Marca')
plt.ylabel('Precio Medio')
plt.title('Comparación del Precio Nuevo y Precio Contado por Marca')
plt.xticks(rotation=45, fontsize=8)
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
df.boxplot(column='precio_contado', by='id_distintivo_ambiental', grid=False)
plt.xlabel('Distintivo Ambiental')
plt.ylabel('Precio Contado')
plt.title('Distribución del Precio Contado por Distintivo Ambiental')
plt.suptitle('')  # Elimina el título automático generado por boxplot
plt.show()

## Visualizaciones

In [ ]:
df.columns

In [ ]:
categorical_columns = ['id_provincia', 'id_concesionario', 'id_distintivo_ambiental','combustible','tipo_cambio', 'id_traccion', 'mes_matriculacion'
               , 'ano_matriculacion','id_sobrealimentacion', 'id_marca', 'id_modelo']
continuous_columns = []

for columna in df.columns: 
    if columna not in categorical_columns: 
        continuous_columns.append(columna)

In [ ]:
def metricas (df, columna):
    
    if columna in categorical_columns:
        
        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 2, 1)
        sns.countplot(data=df, x=columna)
        plt.title(f'Distribución de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df, x=columna, y='precio_contado')
        plt.title(f'Boxplot de {columna} con precio')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()
    
    elif columna in continuous_columns:

        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 3, 1)
        sns.histplot(df[columna], kde=True)
        plt.title(f'Histograma de {columna}')
        
        
        plt.subplot(1, 3, 2)
        sns.boxplot(x=df[columna])
        plt.title(f'Boxplot de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 3, 3)
        sns.scatterplot(x=df[columna], y=df['precio_contado'])
        plt.title(f'Scatterplot de {columna} con precio')
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()

In [ ]:
for columna in continuous_columns:
    metricas(df, columna)

In [ ]:
for columna in categorical_columns:
    metricas(df, columna)

In [ ]:
correlation_matrix = df[continuous_columns].corr()

plt.figure(figsize=(25, 15))

sns.heatmap(correlation_matrix, annot=True)

plt.title('Mapa de Calor de Correlaciones')
plt.show()

## MAPA COROPLÉTICO ##

In [ ]:
mapa_provincias = gpd.read_file('../bin/limites provinciales/recintos_provinciales_inspire_peninbal_etrs89.shp')

In [ ]:
coches_por_provincia = df['id_provincia'].value_counts().reset_index()
coches_por_provincia.columns = ['provincia', 'cantidad']

In [ ]:
def extraer_segundo_nombre(provincia):
    if '/' in provincia:
        return provincia.split('/')[1].strip()
    return provincia

# Aplicar la función al GeoDataFrame para modificar la columna de las provincias
mapa_provincias['NAMEUNIT'] = mapa_provincias['NAMEUNIT'].apply(extraer_segundo_nombre)

In [ ]:
mapa_provincias['NAMEUNIT'] = mapa_provincias['NAMEUNIT'].replace({
    'A Coruña': 'La Coruña',
    'Bizkaia': 'Vizcaya',
    'Ourense': 'Orense',
    'Illes Balears': 'Islas Baleares',
    'Gipuzkoa': 'Guipúzcoa'
})

In [ ]:
nombres_listado = set(df['id_provincia'].unique())
nombres_mapa_actualizados = set(mapa_provincias['NAMEUNIT'].unique())
nombres_unicos_actualizados = nombres_mapa_actualizados - nombres_listado

nombres_unicos_actualizados

In [ ]:
mapa_provincias = mapa_provincias.merge(coches_por_provincia, left_on='NAMEUNIT', right_on='provincia', how='left')

In [ ]:
plt.figure(figsize=(20, 15))
mapa_provincias.plot(column='cantidad', cmap='Oranges', legend=True, edgecolor='black')
plt.title('Cantidad de Coches por Provincia en España')
plt.show()

In [ ]:
mapa_provincias_geojson = mapa_provincias.to_crs("EPSG:4326").to_json()
cantidad_dict = coches_por_provincia.set_index('provincia')['cantidad'].to_dict()

mapa_provincias['cantidad'] = mapa_provincias['NAMEUNIT'].map(cantidad_dict).fillna(0).astype(int)
mapa_provincias['cantidad_log'] = np.log1p(mapa_provincias['cantidad'])
mapa_provincias_geojson = gpd.GeoDataFrame(mapa_provincias).to_crs("EPSG:4326").to_json()

mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

Choropleth(
    geo_data=mapa_provincias_geojson,
    data=mapa_provincias,
    columns=['provincia', 'cantidad_log'],
    key_on='feature.properties.NAMEUNIT',
    fill_color='Oranges',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Cantidad de Coches',
    nan_fill_color='white'
).add_to(mapa)

GeoJson(
    data=mapa_provincias,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAMEUNIT', 'cantidad'],
        aliases=['Provincia:', 'Cantidad de Coches:']
    )
).add_to(mapa)

mapa

In [ ]:
precio_medio_por_provincia = df.groupby('id_provincia')['precio_contado'].mean().reset_index()
precio_medio_por_provincia.columns = ['provincia', 'precio_medio']
#precio_medio_por_provincia['precio_medio'] = precio_medio_por_provincia['precio_medio'].round(2)

precio_medio_dict = precio_medio_por_provincia.set_index('provincia')['precio_medio'].to_dict()

# Añadir el precio medio al GeoDataFrame
mapa_provincias['precio_medio'] = np.round(mapa_provincias['NAMEUNIT'].map(precio_medio_dict).fillna(0), 2)

mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

Choropleth(
    geo_data=mapa_provincias_geojson,
    data=mapa_provincias,
    columns=['NAMEUNIT', 'precio_medio'],
    key_on='feature.properties.NAMEUNIT',
    fill_color='Oranges',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Precio Medio por Provincia',
    nan_fill_color='white'
).add_to(mapa)

GeoJson(
    data=mapa_provincias,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAMEUNIT', 'precio_medio'],
        aliases=['Provincia:', 'Precio Medio:']
    )
).add_to(mapa)

mapa